In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, plot_roc_curve
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from xgboost import XGBClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
training=pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv',index_col=0)
test=pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv',index_col=0)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training.shape

In [ ]:
test.shape

In [ ]:
training.info()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# annot (annotate) parameter
 
plt.figure(figsize=(16,9))
 
sns.heatmap(training.corr(), annot = True)

In [ ]:
hist=training.hist(bins=5,figsize=(12,12))

In [ ]:
training.describe()

In [ ]:
hist=test.hist(bins=5,figsize=(12,12))

In [ ]:
test.info()

In [ ]:
sns.boxplot(training['MonthlyIncome'],showfliers=False)


In [ ]:
training.MonthlyIncome.plot.kde()

In [ ]:
missing_props = training.isna().mean(axis=0)
missing_props

In [ ]:
training['MonthlyIncome'].hist(bins=5)

In [ ]:
training['MonthlyIncome'].fillna(int(training['MonthlyIncome'].mean()), inplace=True)
training['NumberOfDependents'].fillna(int(training['NumberOfDependents'].mean()), inplace=True)


In [ ]:
training.info()

In [ ]:
test['MonthlyIncome'].fillna(int(training['MonthlyIncome'].mean()), inplace=True)
test['NumberOfDependents'].fillna(int(training['NumberOfDependents'].mean()), inplace=True)

In [ ]:
test.info()

In [ ]:
X_train=training.iloc[:,1:]


In [ ]:
y_train=training.iloc[:,0:1]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [ ]:
X_res.shape,y_res.shape

In [ ]:
y_res.hist()

In [ ]:
model = XGBClassifier(n_jobs=-1)
xgb_hyperparams = {
    'clf__max_depth' : np.arange(4,10,1),
    'clf__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'clf__n_estimators' : np.arange(100,200,400),
    'clf__subsample': np.arange(0.5,1,0.1),
    'clf__scale_pos_weight': [10,15,20]
    
}

In [ ]:
'''clf = RandomizedSearchCV(estimator=model,
                         param_distributions=xgb_hyperparams,
                         scoring='roc_auc',
                         n_iter=20,
                         verbose=2)

clf.fit(X_train, y_train)

print("Best parameters:", clf.best_params_)
print("Best score: ", clf.best_score_)'''

In [ ]:
model_fin=XGBClassifier(max_depth=9,
                                   learning_rate=0.01,
                                   n_estimators=800,
                                   subsample=0.9,
                                   scale_pos_weight=10,
                                   n_jobs = -1)


model_fin.fit(X_train, y_train)

In [ ]:
x_test=test.drop(columns=['SeriousDlqin2yrs'])

In [ ]:
y_pred = model_fin.predict_proba(x_test)
y_pred

In [ ]:
df_pred=pd.DataFrame(y_pred[:,1])
df_pred.rename(columns={0:'probability'}, inplace=True)

In [ ]:
df_pred.head()

In [ ]:
df_pred['id'] = range(1,len(df_pred)+1)
df_pred = df_pred.loc[:,['id','probability']]

In [ ]:
df_pred.head()

In [ ]:
df_pred.to_csv('output_predictions.csv', index=False)